In [ ]:
## import required class

from langchain_community.document_loaders import PyPDFLoader ##reads PDF files page by page
import os ##helps to navigate folders and files                                                                                                                   ##Function to Load PDFs

def load_medical_pdfs(pdf_folder_path):
    print(pdf_folder_path)                                                                                                                                        documents=[]                                                                                                                                                                import os
pdf_folder_path = "../data/pdf" 

pdf_files = []

for file_name in os.listdir(pdf_folder_path):
    if file_name.endswith(".pdf"):
        pdf_files.append(file_name)

pdf_files                                                                                                                                                     ## load each pdf

loader = PyPDFLoader("../data/pdf/Antenatal care guidelines.pdf")
pdf_docs = loader.load()
print(pdf_docs)
## to all all pdfs 
all_docs = []
for f in os.listdir("../data/pdf"):
    if f.endswith(".pdf"):
        all_docs.extend(PyPDFLoader(os.path.join("../data/pdf", f)).load())
        print(all_docs)                                 
     documents.extend(pdf_docs)
print(len(documents))
print(f"Total pages loaded: {len(documents)}")
print(documents[0])                                                                                                                              documents = []

for file_name in os.listdir(pdf_folder_path):
    if file_name.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(pdf_folder_path, file_name))
        docs = loader.load()

        for doc in docs:
            doc.metadata["source"] = file_name

        documents.extend(docs)
        documents[0].metadata                                                                                                                  print(f"Total pages loaded: {len(documents)}")                                                                                                from collections import Counter

pdf_summary = Counter(doc.metadata["source"] for doc in documents)

for pdf, pages in pdf_summary.items():
    print(f"{pdf} → {pages} pages")
print(f"Total pages loaded: {len(docs)}")
print(docs[0])                                                                                                                                               !pip install langchain-text-splitters                                                                                                                       from langchain_text_splitters import RecursiveCharacterTextSplitter                                                              text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    
    chunk_overlap=200
)
 chunks = text_splitter.split_documents(documents)                                                                      print(f"Number of chunks created: {len(chunks)}") 
print(chunks[0].page_content[:500])                                                                 
print(chunks[0].page_content[-200:]) 
print(chunks[1].page_content[:200])                                                                                                                       from sentence_transformers import SentenceTransformer 

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

print("Model loaded successfully")                                                                                                                            from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS                                                                                            embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)vectorstore.save_local("faiss_index")
uery = "How is diabetes treated?"

results = vectorstore.similarity_search(query, k=3)

for doc in results:
    print(doc.page_content[:300])
    print(doc.metadata)
    print("-" * 40)
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)faiss_db = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)faiss_db.save_local("faiss_index")
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db"
)
chroma_db.persist()                                                                                                                                              query = "How can we take care a mother after a child birth?" 

results = faiss_db.similarity_search(query, k=3)

for doc in results:
    print(doc.page_content[:200])
    print(doc.metadata)
    print("-" * 40)                                                                                                                                                        pip install langchain langchain-community openai  
 from langchain_openai import ChatOpenAI                                                                                                                                      retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)
## inspect retrievered documents
retrieved_docs = retriever.invoke(
    "What are the treatments for diabetes?"
)

for i, doc in enumerate(retrieved_docs):
    print(f"Doc {i+1}")
    print(doc.page_content[:200])
    print(doc.metadata)
    print("-" * 40)
## duplicate and noise removal
 def remove_duplicates(docs):
    seen = set()
    unique_docs = []

    for doc in docs:
        content = doc.page_content.strip()
        if content not in seen:
            unique_docs.append(doc)
            seen.add(content)

    return unique_docs


clean_docs = remove_duplicates(retrieved_docs)
def filter_by_source(docs, allowed_sources):
    return [
        doc for doc in docs
        if doc.metadata.get("source") in allowed_sources
    ]


trusted_docs = filter_by_source(
    clean_docs,
    allowed_sources=[
        "who_guidelines.pdf",
        "clinical_protocols.pdf"
    ]
)
def build_context(docs, max_chars=3000):
    context = ""
    for doc in docs:
        if len(context) + len(doc.page_content) <= max_chars:
            context += doc.page_content + "\n\n"
    return context


final_context = build_context(trusted_docs)
def safety_check(query):
    restricted_terms = ["dosage", "prescription", "self-medication"]
    for term in restricted_terms:
        if term in query.lower():
            return False
    return True
if not safety_check(query):
    print("⚠️ This question requires a medical professional.")
chatbot_prompt = f"""
You are a healthcare assistant chatbot.
Answer ONLY from the provided context.
If not found, say you don't know.

Context:
{final_context}

Answer the user's question clearly and safely.
"""



